In [ ]:
!pip install ace-tools

In [ ]:
# Step 1: Import required libraries
import pandas as pd

# Step 2: Load the uploaded Excel file to inspect the data
file_path = '/content/drive/MyDrive/BID/ULTIMOS/ecuador/input/datos_ecuador.xlsx'
data = pd.read_excel(file_path, sheet_name=None)  # Load all sheets to inspect

# Step 3: Load the data from the sheet 'Hoja1' to explore its contents
df = data['Hoja1']

# Step 4: Identify non-numeric values in columns 'ha', 'haC', and 'Ton' and replace them with zero
# Convert columns to numeric, forcing errors to NaN, then replace NaN with zero
df['ha'] = pd.to_numeric(df['ha'], errors='coerce').fillna(0)
df['haC'] = pd.to_numeric(df['haC'], errors='coerce').fillna(0)
df['Ton'] = pd.to_numeric(df['Ton'], errors='coerce').fillna(0)

# Step 5: Translate the crop names using the provided dictionary
traducciones = {
    "Palma Africana (Fruta Fresca)": "Oil palm",
    "Maíz": "Corn",
    "Arroz cáscara": "Rice",
    "Soya": "Soy",
    "Plátano (Fruta Fresca)": "Plantain",
    'Caña De Azúcar Para Azúcar (Tallo Fresco)': 'Sugarcane',
    'Caña De Azúcar Para Otros Usos (Tallo Fresco)': 'Sugarcane',
    "Yuca (Raíz Fresca)": "Yuca",
    "Cacao (Almendra Seca)": "Cocoa",
    "MARAÑÓN": "Cashew",
    "Café (Grano Oro)": "Coffee",
    'Sorgo en grano': 'Sorghum',
    'Trigo': 'Wheat',
    'Girasol': 'Sunflower',
    'Papa (Tubérculo Fresco)': 'Potato',
    'Frijol': 'Bean',
    'Alfalfa': 'Alfalfa',
    'Cebada': 'Barley',
    'Maíz Duro Seco (Grano Seco)': 'Corn',
    'Maíz Duro Choclo (En Choclo)': 'Corn',
    'Maíz Suave Choclo (En Choclo)': 'Corn',
    'Maíz Suave Seco (Grano Seco)': 'Corn',
    'Fréjol Seco (Grano Seco)': 'Bean',
    'Fréjol Tierno (En Vaina)': 'Bean',
    'Naranja (Fruta Fresca)': 'Orange',
    'Orito (Fruta Fresca)': 'Orito'
}

df['Cultivo'] = df['Cultivo'].replace(traducciones)

# Step 6: Filter the data for specific departments
departments_to_include = [
    "Azuay", "Chimborazo", "Tungurahua", "Morona Santiago",
    "Orellana", "Sucumbíos", "Zamora Chinchipe", "Napo", "Pastaza"
]
filtered_df = df[df['Departamento'].isin(departments_to_include)]

# Step 7: Filter for the year 2023
df_2023 = filtered_df[filtered_df['Año'] == 2023]

# Step 8: Identify the top 10 crops by area sown (ha) in 2023
top_10_crops_2023 = df_2023.groupby('Cultivo')['ha'].sum().nlargest(10).index

# Step 9: Filter data for the top 10 crops and group by Cultivo to summarize ha, haC, and Ton
top_10_df_2023 = df_2023[df_2023['Cultivo'].isin(top_10_crops_2023)]
summary_table_2023 = top_10_df_2023.groupby('Cultivo').agg({'ha': 'sum', 'haC': 'sum', 'Ton': 'sum'}).reset_index()

# Step 10: Create the first pivot table (by Department and Indicator)
# Create a melted version of the DataFrame to easily restructure by 'Indicador' and 'Cultivo'
melted_df = top_10_df_2023.melt(id_vars=['Departamento', 'Cultivo'], value_vars=['ha', 'haC', 'Ton'], var_name='Indicador', value_name='Valor')

# Pivot the melted DataFrame to create the desired structure
pivot_df = melted_df.pivot_table(index=['Departamento', 'Indicador'], columns='Cultivo', values='Valor', aggfunc='sum').reset_index()

# Fill NaN values with zero to indicate that no production was reported
pivot_df.fillna(0, inplace=True)

# Step 11: Create the second pivot table (by Cultivo and Indicator)
# Melt the DataFrame again but now to prepare for pivoting by 'Cultivo' and 'Indicador'
melted_df_cultivo = top_10_df_2023.melt(id_vars=['Departamento', 'Cultivo'], value_vars=['ha', 'haC', 'Ton'], var_name='Indicador', value_name='Valor')

# Pivot the melted DataFrame to create the desired structure with Cultivo and Indicador as the index
pivot_df_cultivo = melted_df_cultivo.pivot_table(index=['Cultivo', 'Indicador'], columns='Departamento', values='Valor', aggfunc='sum').reset_index()

# Fill NaN values with zero to indicate that no production was reported
pivot_df_cultivo.fillna(0, inplace=True)

# Step 12: Export all three tables to an Excel file with different sheets
output_file_path = '/content/drive/MyDrive/BID/ULTIMOS/ecuador/output/tablas_ecuador.xlsx'

# Create an Excel writer object to save all three tables into different sheets
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    # Write each DataFrame to a different sheet
    summary_table_2023.to_excel(writer, sheet_name='Top_10_Crops_Summary', index=False)
    pivot_df.to_excel(writer, sheet_name='Pivot_By_Department', index=False)
    pivot_df_cultivo.to_excel(writer, sheet_name='Pivot_By_Cultivo', index=False)

# Step 13: Provide the link to download the file
print(f"Download the file: {output_file_path}")


Download the file: /content/drive/MyDrive/BID/ULTIMOS/ecuador/output/tablas_ecuador.xlsx


In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.6 MB/s eta 0:00:00
